In [72]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
import os

In [74]:
from acoustic_knowledge_discovery import extractors

In [75]:
extractor = extractors.Extractor(
    "/home/s.kamboj.400/acoustic_knowledge_discovery/sample_inputs/file.csv", #file metadata csv
    "/home/s.kamboj.400/acoustic_knowledge_discovery/sample_inputs/anno.csv", #annoations csv
    "/home/s.kamboj.400/acoustic_knowledge_discovery/sample_inputs/inputDtory") #input directory with a files subdirectory for audio files & a XC-templates subdirectory for XC templates
kd = extractor.forward()


Generating train split: 2 examples [00:00, 336.51 examples/s]
Generating train split: 2 examples [00:00, 285.45 examples/s]
Casting the dataset: 100%|██████████| 2/2 [00:00<00:00, 562.54 examples/s]

columns in file_ds: ['file_path', ' time_of_day', 'season', 'species']
columns in anno_ds: ['file_path', 'offset_time', 'end_time', 'annotation', 'confidence', 'ID']


In [76]:
#verify properly loaded
for row in kd.anno_ds:
    print(row)

{'file_path': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', 'offset_time': 0.0, 'end_time': 5.0, 'annotation': 'owl', 'confidence': 0.9, 'ID': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_0'}
{'file_path': 'XC758442 - Common Blackbird - Turdus merula.wav', 'offset_time': 5.0, 'end_time': 7.0, 'annotation': 'bat', 'confidence': 0.8, 'ID': 'XC758442 - Common Blackbird - Turdus merula.wav_5'}


In [77]:
from acoustic_knowledge_discovery import features

THRESHOLD = 0.6 # Customizable threshold for detections

template_matching= features.TemplateMatching(
    os.path.join(extractor.base_dir, "files"), # path to audio clips
    os.path.join(extractor.base_dir, "XC-templates"), # path to XC templates
    THRESHOLD, 
    extractor.anno_ds) #dataset with annotations to append results to

new_kd= template_matching()

1 total templates
Filtered down to 1 unique species templates
  XC758442 - Common Blackbird - Turdus merula: XC758442 - Common Blackbird - Turdus merula.wav
Progress: 1/1 templates completed (100.0%)
Total matches found above the threshold of 0.6: 1


In [78]:
#verify template matching results properly loaded
for row in new_kd:
    print(row)

{'file_path': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3', 'offset_time': 0.0, 'end_time': 5.0, 'annotation': 'owl', 'confidence': 0.9, 'ID': 'XC233970 - Black-chinned Sparrow - Spizella atrogularis.mp3_0'}
{'file_path': 'XC758442 - Common Blackbird - Turdus merula.wav', 'offset_time': 5.0, 'end_time': 7.0, 'annotation': 'bat', 'confidence': 0.8, 'ID': 'XC758442 - Common Blackbird - Turdus merula.wav_5'}
{'file_path': 'XC758442 - Common Blackbird - Turdus merula.wav', 'offset_time': 0.046439909297052155, 'end_time': 5.14, 'annotation': 'XC758442 - Common Blackbird - Turdus merula', 'confidence': 0.6061245203018188, 'ID': 'XC758442 - Common Blackbird - Turdus merula.wav_0.046439909297052155'}
